# Food Recommendations

Below is the output of a recommender system for healthy foods.  

In [2]:
# Imports and settings
import pandas as pd
from Recommender_system.Recommender import UserRatings, collabFilteringModel, getUserFavFoods, getMealRec, format_meal_rec

pd.options.display.max_colwidth = 500

In [3]:
# Load dataset
user_food_table = pd.read_csv("Recommender_system/Dataset.csv")

### Set User and Meal Kind

Here, set the user ID and kind of meal (`'breakfast'`, `'lunch'`, `'dinner'`, or `''` for all meals)

In [4]:
user_id = 1665
meal_kind = 'breakfast'

## 1. Summary of User's Meals

Below are the food items in the user's food log for this meal type

In [5]:
user_ratings = UserRatings(user_food_table, meal_kind)
user_food_list, meal_summary = user_ratings.mealSummary(user_id)
meal_summary

,list of foods
meal_id,
2012,"oatmeal, strawberries, oats"
2015,"cheerios cereal, low fat milk"
2042,"avocado, peanut butter, strawberry preserves, thin bagel, peanut butter, strawberry preserves, avocado"
2048,"egg, biscuit, eggs, french fries, raisin"
2052,"sausage, bagel"
2055,"corn, sweet potato, butter"
2060,"hot cereal, cream of wheat, cinnamon"
2064,"sandwich, turkey, bread"
2097,"cereal, milk"


## 2. Recommending food items

In [6]:
# Run and fit the food recommendation model
food_rec_model = collabFilteringModel(d=8, sigmasq=0.6, lambd=1, nUsers=user_ratings.nUsers, nFoods=user_ratings.nFoods)
food_rec_model.fit(user_ratings.ratings_train.values,seed_u=3,seed_v=5) #set the seed to get reproducible results

collabFilteringModel(d=8, lambd=1, nFoods=410, nUsers=118, sigmasq=0.6)

In [7]:
# Get favorite and least favorite foods
fav_foods = getUserFavFoods(user_id, user_ratings, food_rec_model)
fav_foods = fav_foods[~fav_foods.food.isin(user_food_list)]

In [8]:
top_fav_foods = fav_foods.iloc[0:10,].food
top_fav_foods = top_fav_foods.sample(5, random_state=123)

In [9]:
least_fav_foods = fav_foods.iloc[-10:-1,].food
least_fav_foods = least_fav_foods.sample(5, random_state=123)

Here are 10 food items - based on what the user has eaten before, please rate the extent to which you think the user would or would not like each food item on a scale from 1, the user would not like this food item at all, to 7, the user would like this food item a lot.

In [28]:
test_food_list = pd.DataFrame({'label': 'fav', 'food': top_fav_foods})
test_food_list = test_food_list.append(pd.DataFrame({'label': 'least_fav', 'food': least_fav_foods})).sample(frac=1).reset_index()
test_food_list['rating'] = '1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)'
test_food_list[['food','rating']]

,food,rating
0,fage,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
1,coffee with milk,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
2,granny smith apple,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
3,rice cakes,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
4,chicken,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
5,non dairy creamer,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
6,hemp seeds,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
7,bacon,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
8,muesli,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
9,canadian bacon,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)


## 3. Recommending complete meals

In [13]:
association_rules = user_ratings.calculateAssociationRules(0.005)

In [22]:
seed_food = 'yogurt'

In [23]:
meal_rec = getMealRec(user_id, user_ratings=user_ratings, model=food_rec_model, seed_food=seed_food)
suggestion = meal_rec.iloc[0,:]

In [18]:
rec = format_meal_rec(suggestion, seed_food)
rec

'To go with 2 servings of yogurt, try 0.5 serving of high protein oatmeal'

How would you rate this recommendation on a scale from 1 to 7?

`1 - 2 - 3 - 4 - 5 - 6 - 7`

In [26]:
seed_food = 'peanut butter'
meal_rec = getMealRec(user_id, user_ratings=user_ratings, model=food_rec_model, seed_food=seed_food)
suggestion = meal_rec.iloc[0,:]
rec = format_meal_rec(suggestion, seed_food)
rec

'To go with 2 servings of peanut butter, try 1 serving of coffee with half and half and pancakes'

How would you rate this recommendation on a scale from 1 to 7?

`1 - 2 - 3 - 4 - 5 - 6 - 7`